In [1]:
import torch 
import torchvision 


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu" 

In [3]:
def set_seed(seed=42): 
    torch.manual_seed(seed) 
    torch.cuda.manual_seed(seed)

In [4]:
from pathlib import Path 

image_path = Path("data/pizza_steak_sushi/")

train_dir = image_path / "train" 
test_dir = image_path / "test" 

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [5]:
from torchvision import transforms 

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
    # transforms.ToTensor()
])

manual_transforms 

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [15]:
auto_transforms = weights.transforms() 
auto_transforms 

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [16]:
from going_modular import data_setup 

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir, test_dir, auto_transforms, 32)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x22a34e7fe50>,
 ['pizza', 'steak', 'sushi'])

In [17]:
import torchvision

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT 

model = torchvision.models.efficientnet_b0(weights=weights).to(device) 
model.classifier 

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [18]:
for param in model.features.parameters(): 
    param.requires_grad = False 
    

In [19]:
from torch import nn 

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True), 
    nn.Linear(in_features=1280, out_features=len(class_names)) 
).to(device) 

In [20]:
from torchinfo import summary 

summary(model, 
        input_size=(32, 3, 224, 224), 
        verbose=0, 
        col_names=["input_size", "output_size", "num_params", "trainable"], 
        col_width=20, 
        row_settings=["var_names"]
        )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [21]:
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

In [22]:
from torch.utils.tensorboard import SummaryWriter 

writer = SummaryWriter() 
writer 

In [23]:
from going_modular.engine import train_step, test_step 

def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):  
   
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []} 
    
    for epoch in range(epochs): 
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device) 
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device) 
        
        print(f"Epoch: {epoch+1} | "
              f"train_loss: {train_loss:.4f} | " 
              f"train_acc: {train_acc:.4f} | "
              f"test_loss: {test_loss:.4f} | "
              f"test_acc: {test_acc:.4f}") 
        
        results["train_loss"].append(train_loss) 
        results["train_acc"].append(train_acc) 
        results["test_loss"].append(test_loss) 
        results["test_acc"].append(test_acc)  
        
        
        writer.add_scalars(
            main_tag="Loss", 
            tag_scalar_dict={"train_loss": train_loss, 
                             "test_loss": test_loss}, 
            global_step=epoch) 
        
        writer.add_scalars(
            main_tag="Accuracy", 
            tag_scalar_dict={"train_acc": train_acc, 
                             "test_acc": test_acc}, 
            global_step=epoch) 
        
        writer.add_graph(model=model, 
                         input_to_model=torch.randn(32, 3, 244, 244).to(device)) 
        
    writer.close()  
        
    return results 

In [24]:
set_seed() 
results = train(model=model, 
                train_dataloader=train_dataloader, 
                test_dataloader=test_dataloader, 
                optimizer=optimizer, 
                loss_fn=loss_fn, 
                epochs=5, 
                device=device)

Epoch: 1 | train_loss: 1.1296 | train_acc: 0.3438 | test_loss: 0.9155 | test_acc: 0.7008
Epoch: 2 | train_loss: 0.9120 | train_acc: 0.6406 | test_loss: 0.8147 | test_acc: 0.7746
Epoch: 3 | train_loss: 0.8178 | train_acc: 0.7461 | test_loss: 0.6645 | test_acc: 0.8655
Epoch: 4 | train_loss: 0.7223 | train_acc: 0.7500 | test_loss: 0.6882 | test_acc: 0.8248
Epoch: 5 | train_loss: 0.6375 | train_acc: 0.7695 | test_loss: 0.6551 | test_acc: 0.8655


In [29]:
# %load_ext tensorboard
# # %reload_ext tensorboard 
# %tensorboard --logdir runs 

In [30]:
import os 
from datetime import datetime 

def create_writer(experiment_name, model_name, extra): 
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance tracking to a specific experiment"""
    timestamp = datetime.now().strftime("%y-%m-%d") 
    
    if extra: 
        log_dir = os.path.join("runs", timestamp, experiment_name, extra)
    else: 
        log_dir = os.path.join("runs", timestamp, experiment_name) 
        
    print(f"[INFO] Created SummaryWriter saving to {log_dir}")
        
    return SummaryWriter(log_dir=log_dir) 

In [ ]:
def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device, writer):  
   
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []} 
    
    for epoch in range(epochs): 
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device) 
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device) 
        
        print(f"Epoch: {epoch+1} | "
              f"train_loss: {train_loss:.4f} | " 
              f"train_acc: {train_acc:.4f} | "
              f"test_loss: {test_loss:.4f} | "
              f"test_acc: {test_acc:.4f}") 
        
        results["train_loss"].append(train_loss) 
        results["train_acc"].append(train_acc) 
        results["test_loss"].append(test_loss) 
        results["test_acc"].append(test_acc)  
        
        
        if writer: 
            writer.add_scalars(
                main_tag="Loss", 
                tag_scalar_dict={"train_loss": train_loss, 
                                "test_loss": test_loss}, 
                global_step=epoch) 
            
            writer.add_scalars(
                main_tag="Accuracy", 
                tag_scalar_dict={"train_acc": train_acc, 
                                "test_acc": test_acc}, 
                global_step=epoch) 
            
            writer.add_graph(model=model, 
                            input_to_model=torch.randn(32, 3, 244, 244).to(device)) 
        
            writer.close()  
        else: 
            pass 
        
    return results 